In [4]:
from geopy.geocoders import Nominatim
from openrouteservice.directions import directions
import openrouteservice as ors
import folium
from functools import reduce
import IPython
import time
import pandas as pd
import numpy as np
from typing import Tuple, List

from itertools import combinations
import random

In [3]:
from utils import coordinates_dict, get_kms, get_coordinates_route

In [43]:
geolocator = Nominatim(user_agent="my_application")

In [44]:
client = ors.Client(key="5b3ce3597851110001cf6248a23ed70a99d1426492e40e4adca6bd8e")

# Store the distances between the adresses

In [45]:
# Nearest neighbor optimization
list_of_locations = [
    "travessera de dalt 32, Barcelona, Spain", #Sortida
    "carrer de la cova del drac 6, Matadepera, Spain",
    "avinguda diagonal 449, Barcelona, Spain",
    "carrer balmes 18, Matadepera, Spain",
    "rambla egàra 17, Terrassa, Spain",
    "carrer de Nicolau Talló 85, Terrassa, Spain",
    "carrer de Valentí Almirall 40, Sabadell, Spain",
    "carrer de la Salut 42, Sabadell, Spain",
    "carrer can manent 1, Sabadell, Spain",
    "carrer de les Pedritxes 28, Matadepera, Spain",
    "carrer de la Carossa 22, Matadepera, Spain",
    "ronda de Llevant 30, Castellar del Vallès, Spain",
    "Carrer de la Riera 11, Castellar del Vallès, Spain",
    "Carrer de l'Església 22, Castellar del Vallès, Spain",
    "avinguda Pau Casals 86, Sant Quirze del Vallès, Spain",
    "avinguda Can Pallàs 4, Sant Quirze del Vallès, Spain",
    "carrer de Sant Ramón 180, Cerdanyola del Vallès, Spain",
    "avinguda de la Primavera 74, Cerdanyola del Vallès, Spain",
    "Carrer de Balmes 16, Barcelona, Spain",
    "Passeig de Gracia 92, Barcelona, Spain",
    "Plaça de Catalunya, Barcelona, Spain",
    "Carrer de Provença 261, Barcelona, Spain",
    "Carrer de Tuset 13, Barcelona, Spain",
    "Avinguda Diagonal 555, Barcelona, Spain",
    "Rambla de Catalunya 97, Barcelona, Spain",
    "Passeig de Sant Joan 82, Barcelona, Spain",
    "Carrer del Comte d'Urgell 232, Barcelona, Spain",
    "Carrer de Muntaner 262, Barcelona, Spain",
    "Passeig del Pla 48, Matadepera, Spain"
]

# Terrassa adresses:
terrassa_coordinates = [
    (41.56509893937577, 2.0074291595580838), # "Rambla d'Ègara, 270, 08221 Terrassa, Barcelona"
    (41.56255144660603, 2.022511482022627), # "Carrer de Colom, 34, 08222 Terrassa, Barcelona"
    (41.563186210767405, 2.0272192200010015), # "Av. de Barcelona, 111, 08222 Terrassa, Barcelona"
    (41.56039167041793, 2.0170786874117153), # "Carrer de Topete, 84, 08221 Terrassa, Barcelona"
    (41.56630143443987, 2.0128793991198433), # "Plaça de Mossèn Jacint Verdaguer, 16, 08221 Terrassa, Barcelona"
    (41.570944260548245, 2.023022050098688), #"Av. de Jaume I, 91, 08227 Terrassa, Barcelona"
    (41.558339703887796, 2.0291680060971915), #"Carretera de Montcada, 598, 08223 Terrassa, Barcelona"
    (41.55839952475521, 2.0296691321594107), # Carretera de Montcada, 602, 08223 Terrassa, Barcelona
    (41.556368357820546, 2.0311828199081585), # Carrer del Duero, 17, 08223 Terrassa, Barcelona
    (41.55604464485819, 2.030190110107441), # "C. del Miño, 25, 08223 Terrassa, Barcelona"
    (41.55061752205613, 2.0227542709780892), # "Avinguda de Can Jofresa, 3, 08223 Terrassa, Barcelona",
    (41.56010715292179, 2.0233373447537493), # "Carrer de Colom, 104, 08222 Terrassa, Barcelona",
    (41.56269658475855, 2.022321985324917), # "Carrer de Colom, 24, 08222 Terrassa, Barcelona",
    (41.561896996632235, 2.0203822486813703), # Carrer del Prior Tàpia, 5, 08222 Terrassa, Barcelona
    (41.56125925132496, 2.038785166679293), #"Carrer de Palència, 9, 08227 Terrassa, Barcelona",
    (41.54217806099222, 1.9973211159677866), #"Camí de les Martines, 128, 08224 Barcelona"
    (41.5419124783373, 1.9925196456832686), # Passatge dels Quatre Vents, 1, 08191 Rubí, Barcelona
    (41.559730162695, 2.0064082467330007), # C. d'Arquímedes, 120, 08224 Terrassa, Barcelona
    (41.56049081193857, 2.006493406864138), # Carrer de Gutenberg, 35, 08224 Terrassa, Barcelona
    (41.56206652154605, 2.00893254068073), # Carrer d'Iscle Soler, 7, 08221 Terrassa, Barcelona
    (41.562473926914386, 2.0096352794356296), # Carrer de Joan Coromines, 8, 10, 08221 Terrassa, Barcelona
    (41.564398531349084, 2.0104184844818493), # Carrer de la Rasa, 54, 08221 Terrassa, Barcelona
    (41.57711606796531, 2.0266629252054944), # Rambla de Francesc Macià, 236, 08226 Terrassa, Barcelona
    (41.57776615832105, 2.0214057956662845), # C. del Doctor Ferran, 164, 08226 Terrassa, Barcelona
    (41.583945702980714, 2.016652921125985), # Carrer de Granollers, 1, 08225 Terrassa, Barcelona
    (41.58267774074262, 2.010687688321248) # Carrer de Ferran Casablancas, 12, 08225 Terrassa, Barcelona
]


sant_cugat = [
    "Plaça dels Quatre Cantons, 3, 08172 Sant Cugat del Vallès, Barcelona",
    "Carrer de Manel Farrés, 39, 08173 Sant Cugat del Vallès, Barcelona",
    "Carrer Caldes i Arús, 1, 08172 Sant Cugat del Vallès, Barcelona",
    "Carrer de Sant Antoni, 56, 08172 Sant Cugat del Vallès, Barcelona",
    "Camí Sant Cugat al Papiol, 31, 08195 Sant Cugat del Vallès, Barcelona"
]

list_coordinates = [
    (41.6056790858793, 2.0174380771279825), #Bikemola
    (41.58379798711553, 2.008765797482567), #Institut Serra de l'Obac Terrassa
    (41.54334575136443, 1.9911809971232837), #Restaurant 4 vents
    (41.477945427273355, 2.0724103097253446) # Bonaparte Sant Cugat
]

In [46]:
def create_dist_matrix(coordinates):    

    matrix = pd.DataFrame(columns=coordinates, index=coordinates)

    for i, adress1 in enumerate(matrix.index):
        print("row" + str(i) + "done")
        for j, adress2 in enumerate(matrix.columns):
        
            if (adress1 == adress2):
                matrix.loc[adress1, adress2] = 0
            
            else:
                adress1_dict = coordinates_dict(adress1)
                adress2_dict = coordinates_dict(adress2)
                
                route_params = {
                    'coordinates': [
                        [adress1_dict["longitude"], adress1_dict["latitude"]], 
                        [adress2_dict["longitude"], adress2_dict["latitude"]]
                    ],
                    'format': 'geojson',
                    'profile': 'driving-car',
                    'preference': 'fastest',
                    'units': 'km'
                }

                route = client.directions(**route_params)
                distance = get_kms(route)
                
                matrix.loc[adress1, adress2] = distance

    return matrix

In [47]:
str_terrassa_coordinates = [str(elem) for elem in terrassa_coordinates]

terrassa_matrix = create_dist_matrix(str_terrassa_coordinates)

row0done
row1done


/home/leibniz/anaconda3/envs/rec/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/leibniz/anaconda3/envs/rec/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/leibniz/anaconda3/envs/rec/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/leibniz/anaconda3/envs/rec/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


row2done
row3done
row4done
row5done


/home/leibniz/anaconda3/envs/rec/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 9th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


row6done
row7done
row8done
row9done
row10done
row11done
row12done
row13done
row14done
row15done
row16done
row17done
row18done
row19done
row20done
row21done
row22done
row23done
row24done
row25done


In [48]:
terrassa_matrix.to_csv("terrassa_matrix.csv")

In [49]:
terrassa_matrix = pd.read_csv("terrassa_matrix.csv")

In [50]:
terrassa_matrix.set_index("Unnamed: 0", inplace=True)

# Neighbour optimization

In [24]:
def optimize_route_matrix(distances_matrix: pd.DataFrame) -> list:    

    locations = list(distances_matrix.columns.copy())
    optimal_route = []

    current_location = locations[0]
    remaining = locations.copy()
    remaining.remove(current_location)

    while remaining:

        distances_to_current_location = distances_matrix.loc[current_location, remaining]

        print("CURRENT LOCATION:")
        print(current_location)
        print(type(current_location))
        print("REMAINING:")
        print(remaining)
        #Keep this one to romve it from remaining
        closest_location = distances_to_current_location.astype(float).idxmin()

        current_location_dict = coordinates_dict(current_location)
        closest_location_dict = coordinates_dict(closest_location)

        route_params = {
            'coordinates': 
                [
                    [current_location_dict["longitude"], current_location_dict["latitude"]], 
                    [closest_location_dict["longitude"], closest_location_dict["latitude"]]
                ],
            'format': 'geojson',
            'profile': 'driving-car',
            'preference': 'fastest',
            'units': 'km'
        }
        optimal_route.append(client.directions(**route_params))
        
        remaining.remove(closest_location)

        #To continue iterating
        current_location = closest_location

    return optimal_route

In [61]:
def generate_polyline_colors(n):
    import random
    
    colors = []
    for i in range(n):
        color = "#{:06x}".format(random.randint(0, 0xFFFFFF))
        colors.append(color)
    
    return colors

def generate_darker_polyline_colors(n):
    colors = []
    for i in range(n):
        # Generate a random color with maximum brightness (i.e., 0xFF)
        red = random.randint(0, 255)
        green = random.randint(0, 255)
        blue = random.randint(0, 255)
        
        # Compute the average brightness of the color
        brightness = (red + green + blue) / 3
        
        # Reduce the brightness to generate a darker color
        dark_factor = random.uniform(0.5, 0.8)  # adjust this factor to control how dark the colors are
        red = int(red * dark_factor)
        green = int(green * dark_factor)
        blue = int(blue * dark_factor)
        
        # Convert the color components to hexadecimal format and append to the colors list
        color = "#{:02x}{:02x}{:02x}".format(red, green, blue)
        colors.append(color)
    
    return colors

def generate_routemap(optimal_route):
    maps = []

    optimal_map = folium.Map(location=list([41.56509893937577, 2.0074291595580838]), titles="stamentoner", zoom_start=13)
    colors = generate_darker_polyline_colors(len(optimal_route))

    for i in range(len(optimal_route)):
        folium.PolyLine(locations=[list(reversed(cord)) for cord in get_coordinates_route(optimal_route[i])], color=colors[i]).add_to(optimal_map)
        
        IPython.display.clear_output(wait=True)
        display(optimal_map)
        time.sleep(1)

        maps.append(optimal_map)
    
    return maps

In [30]:
optimal_route = optimize_route_matrix(terrassa_matrix)

CURRENT LOCATION:
(41.56509893937577, 2.0074291595580838)
<class 'str'>
REMAINING:
['(41.56255144660603, 2.022511482022627)', '(41.563186210767405, 2.0272192200010015)', '(41.56039167041793, 2.0170786874117153)', '(41.56630143443987, 2.0128793991198433)', '(41.570944260548245, 2.023022050098688)', '(41.558339703887796, 2.0291680060971915)', '(41.55839952475521, 2.0296691321594107)', '(41.556368357820546, 2.0311828199081585)', '(41.55604464485819, 2.030190110107441)', '(41.55061752205613, 2.0227542709780892)', '(41.56010715292179, 2.0233373447537493)', '(41.56269658475855, 2.022321985324917)', '(41.561896996632235, 2.0203822486813703)', '(41.56125925132496, 2.038785166679293)', '(41.54217806099222, 1.9973211159677866)', '(41.5419124783373, 1.9925196456832686)']
CURRENT LOCATION:
(41.56630143443987, 2.0128793991198433)
<class 'str'>
REMAINING:
['(41.56255144660603, 2.022511482022627)', '(41.563186210767405, 2.0272192200010015)', '(41.56039167041793, 2.0170786874117153)', '(41.57094426054

In [37]:
generate_routemap(optimal_route)

# 2-OPT optimization

In [55]:
def route_distance(route, route_matrix):
    #Calculates the distance of a given route

    total_distance = 0
    
    for i in range(1, len(route)):

        total_distance += route_matrix.loc[route[i-1], route[i]]
    
    return total_distance

def swap_nodes(route, i, j):
    # Swaps two of the nodes of the route

    new_route= route.copy()
    node_position_i = new_route[i]
    node_position_j = new_route[j]

    new_route[i] = node_position_j
    new_route[j] = node_position_i

    return new_route

def two_optimization(route, route_matrix, max_iters):    
    # Performs 2-opt optimization algorithm on the routes. The format of the route must be a list of the coordinates in string format
    best_route = route.copy()

    improvement = True
    while improvement:

        improvement = False
        #We get all the posible combinations of positions to pass it to the function swap_nodes
        positions = range(1, len(best_route))
        possible2swaps = list(combinations(positions, 2))

        for i, j in possible2swaps:
            new_route = swap_nodes(best_route, i, j)

            if route_distance(new_route, route_matrix) < route_distance(best_route, route_matrix):

                improvement = True

                best_route = new_route
    
    return best_route

def get_kms_route(route):
    # Gets the total distance of the list of subroutes
    # For example the route that returns the function optimize_route_matrix

    total_kms = 0
    for subroute in route:
        total_kms += get_kms(subroute)

    return total_kms

In [56]:
str_terrassa_coordinates[0]

'(41.56509893937577, 2.0074291595580838)'

In [21]:
better_route = two_optimization(
    route = str_terrassa_coordinates,
    route_matrix = terrassa_matrix,
    max_iters = 300
)

Better route found! :
['(41.56509893937577, 2.0074291595580838)', '(41.563186210767405, 2.0272192200010015)', '(41.56255144660603, 2.022511482022627)', '(41.56039167041793, 2.0170786874117153)', '(41.56630143443987, 2.0128793991198433)', '(41.570944260548245, 2.023022050098688)', '(41.558339703887796, 2.0291680060971915)', '(41.55839952475521, 2.0296691321594107)', '(41.556368357820546, 2.0311828199081585)', '(41.55604464485819, 2.030190110107441)', '(41.55061752205613, 2.0227542709780892)', '(41.56010715292179, 2.0233373447537493)', '(41.56269658475855, 2.022321985324917)', '(41.561896996632235, 2.0203822486813703)', '(41.56125925132496, 2.038785166679293)', '(41.54217806099222, 1.9973211159677866)', '(41.5419124783373, 1.9925196456832686)']
Better route found! :
['(41.56509893937577, 2.0074291595580838)', '(41.56039167041793, 2.0170786874117153)', '(41.56255144660603, 2.022511482022627)', '(41.563186210767405, 2.0272192200010015)', '(41.56630143443987, 2.0128793991198433)', '(41.5709

# Random optimization

### Generate n random routes and keep the best one

In [22]:
def generate_n_random(n, lst):
    
    shuffles = []

    for i in range(n):
        
        shuffled_lst = lst.copy()
        random.shuffle(shuffled_lst)

        shuffles.append(shuffled_lst)
    
    return shuffles

In [23]:
def random_optimization(n, route_matrix):

    # We eliminate the first one because we want the elemenbt to be kept fixed
    best_route = list(route_matrix.columns[1:])
    random_routes = generate_n_random(n, best_route)

    for r in random_routes:
        if route_distance(r, route_matrix) < route_distance(best_route, route_matrix):
            print("Better route found !")

            best_route = r
    
    return [route_matrix.columns[0]] + best_route

In [24]:
random_route = random_optimization(
    n = 100000,
    route_matrix = terrassa_matrix
)

Better route found !
Better route found !
Better route found !


In [26]:
print("Non-optimized route: " + str(route_distance(str_terrassa_coordinates, terrassa_matrix)))
print("Optimized route 2-opt: " + str(route_distance(better_route, terrassa_matrix)))
print("Random optimization: " + str(route_distance(random_route, terrassa_matrix)))

Non-optimized route: 25.602
Optimized route 2-opt: 17.039
Random optimization: 24.483


### Test to get get random inicializations for the input of 2-opt and get the best route:

In [63]:
initial_route = list(terrassa_matrix.columns[1:])
starting_point = [terrassa_matrix.columns[0]]

# Generate n random routes:
random_routes = generate_n_random(500, initial_route)
random_routes_optimized = []

for random_route in random_routes:

    better_route = two_optimization(
        route = starting_point + random_route,
        route_matrix = terrassa_matrix,
        max_iters = 300
    )

    random_routes_optimized.append(better_route)

# Select the most optimal route of the optimized random routes:
kms_random_routes_optimized = [route_distance(r, terrassa_matrix) for r in random_routes_optimized]

random_best_route_kms = min(kms_random_routes_optimized)
random_best_route_index = kms_random_routes_optimized.index(random_best_route_kms)

random_best_route = random_routes_optimized[random_best_route_index]

print("Best route: ")
print("-----Kms: " + str(random_best_route_kms))
print("-----Index: " + str(random_best_route_index))
print("ROUTE:")

print(random_best_route)

Best route: 
-----Kms: 27.441
-----Index: 38
ROUTE:
['(41.56509893937577, 2.0074291595580838)', '(41.58267774074262, 2.010687688321248)', '(41.583945702980714, 2.016652921125985)', '(41.57776615832105, 2.0214057956662845)', '(41.57711606796531, 2.0266629252054944)', '(41.570944260548245, 2.023022050098688)', '(41.556368357820546, 2.0311828199081585)', '(41.55604464485819, 2.030190110107441)', '(41.558339703887796, 2.0291680060971915)', '(41.55839952475521, 2.0296691321594107)', '(41.56125925132496, 2.038785166679293)', '(41.563186210767405, 2.0272192200010015)', '(41.56269658475855, 2.022321985324917)', '(41.56255144660603, 2.022511482022627)', '(41.561896996632235, 2.0203822486813703)', '(41.56039167041793, 2.0170786874117153)', '(41.559730162695, 2.0064082467330007)', '(41.56049081193857, 2.006493406864138)', '(41.564398531349084, 2.0104184844818493)', '(41.56630143443987, 2.0128793991198433)', '(41.562473926914386, 2.0096352794356296)', '(41.56206652154605, 2.00893254068073)', '(41.

# Validations of models and tests

### We want to visualize the route optimized with 2-opt

In [32]:
def generate_georoute(str_route):
    # Takes a list with the coordinates in order of the route and returns a calculated route with openroute service

    geolocated_route = []

    for i in range(len(str_route)-1):
        coordinate1 = coordinates_dict(str_route[i])
        coordinate2 = coordinates_dict(str_route[i+1])

        route_params = {
            'coordinates': 
                [
                    [coordinate1["longitude"], coordinate1["latitude"]], 
                    [coordinate2["longitude"], coordinate2["latitude"]]
                ],
            'format': 'geojson',
            'profile': 'driving-car',
            'preference': 'fastest',
            'units': 'km'
        }
        geolocated_route.append(client.directions(**route_params))

    return geolocated_route

In [160]:
better_geolocated = generate_georoute(better_route)

In [65]:
generate_routemap(better_geolocated)

NameError: name 'better_geolocated' is not defined

In [66]:
random_geolocated = generate_georoute(random_best_route)
generate_routemap(random_geolocated)

# Deprecated functions

In [ ]:
#DEPRECATED

"""
def optimize_route(list_geolocated: list) -> list:    
    optimal_route = []
    current_location = list_geolocated[0]

    for i in range(len(list_geolocated)-1):

        list_geolocated.remove(current_location)

        possible_routes = []

        for adress in list_geolocated:

            route_params = {
                'coordinates': [[current_location.longitude, current_location.latitude], [adress.longitude, adress.latitude]],
                'format': 'geojson',
                'profile': 'driving-car',
                'preference': 'fastest',
                'units': 'km'
            }
            possible_routes.append((client.directions(**route_params), adress))


        # Get the km from all the possible routes from current_location to other_locations:
        kms_routes = [(get_kms(r[0]), r[1]) for r in possible_routes]
        kms = [r[0] for r in kms_routes]

        closest_route_idx = kms.index(min(kms))

        best_route = possible_routes[closest_route_idx]

        optimal_route.append(best_route)

        current_location = best_route[1]

        print(best_route[1])

    return optimal_route
"""